In [1]:
from typing import List, Tuple, Union
from datetime import datetime, timedelta
import math
import os
import torch
from torch import nn
from torch.nn import functional as F
from torch_geometric.data import Data, InMemoryDataset
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, GATConv, SAGEConv, AGNNConv
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from tqdm import tqdm
from argparse import Namespace
import gc
import wandb
import random
# os.environ['CUDA_LAUNCH_BLOCKING']='1'
# os.environ['TORCH_USE_CUDA_DSA']='enable'

In [2]:
def all_seed(seed=6666):
    np.random.seed(seed)
    random.seed(seed)
    # CPU
    torch.manual_seed(seed)
    # GPU
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.cuda.manual_seed(seed)
    # python全局
    os.environ['PYTHONHASHSEED'] = str(seed)
    # cudnn
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.enabled = False
    print(f'Set env random_seed = {seed}')

all_seed(2023)

Set env random_seed = 2023


# loading Data

In [3]:
df = pd.read_csv('./data/train.csv')
test_df = pd.read_csv('./data/test.csv')
edges = pd.read_csv('./data/edges.csv')
# 查看test 是否全没有出现
test_no_see_count = len(set(test_df['id'].unique()) - set(df['id'].unique()))
test_no_see_count == test_df['id'].nunique()

# 查看test是否在 edge中出现
no_edge_count = len(
    set(edges['id_1'].tolist() + edges['id_2'].tolist()) - 
    set(test_df['id'].unique()) - 
    set(df['id'].unique())
)


total_node = sorted(set(edges['id_1'].tolist() + edges['id_2'].tolist()))
lb_ecoder = LabelEncoder()
lb_ecoder.fit(np.array(total_node))
edges['id_1_idx'] = lb_ecoder.transform(edges['id_1'])
edges['id_2_idx'] = lb_ecoder.transform(edges['id_2'])


label_df = pd.DataFrame(np.array(total_node), columns=['id'])
label_df['id_idx'] = lb_ecoder.transform(label_df['id'])
label_df = label_df.merge(df, how='left', on='id')
label_df['target'] = label_df['target'].fillna(-1)


# dataLoader

In [4]:

class graphData(InMemoryDataset):
    def __init__(self, root, label_df_in, data_type='train', sample_method='randomEdge', random_sample_ratio=0.3, random_graph_count=1024):
        self.idx = {
            'train': 0, 'val': 1, 'test': 2
        }[data_type]
        self.sample_method = sample_method
        label_df_in = label_df_in.copy(deep=True).sort_values(by='id_idx', ignore_index=True)
        self.mask = torch.tensor(label_df_in['target'] != -1)
        self.node_features = torch.LongTensor(label_df_in['id_idx'].values)
        self.node_label = torch.tensor(label_df_in['target'].values).float()
        self.node_info = label_df_in['id'].values
        self.random_sample_ratio = random_sample_ratio
        self.random_graph_count = random_graph_count
        super().__init__(root)
        self.data, self.slices = torch.load(self.processed_paths[self.idx])

    @property
    def processed_file_names(self):
        return ['edge_processed__tr.dataset', 'edge_processed__val.dataset', 'edge_processed__test.dataset']

    def similar_sample(self, sample_df, sample_node=300):
        def agg_rand(l):
            return np.random.choice(l, size=1)[0]
        
        rand_node = np.random.choice(sample_df['id_1_idx'].unique().tolist(), sample_node, replace=False).tolist()
        if self.idx == 2:
            rand_node = self.node_features[self.node_label == 1].cpu().detach().numpy()
            rand_node = np.random.choice(rand_node.tolist(), sample_node//2, replace=False).tolist()
            sample_node_ = sample_node - sample_node // 2 
            rand_node_ = np.random.choice(sample_df['id_1_idx'].unique().tolist(), sample_node_, replace=False).tolist()
            rand_node = list(set(rand_node) | set(rand_node_))
            

        pos_node_df = sample_df.loc[sample_df.id_1_idx.isin(rand_node), ['id_1_idx', 'id_2_idx']]
        sample_node = pos_node_df.id_1_idx.nunique()
        pos_node_df = pos_node_df.groupby('id_1_idx', as_index=False).agg(
            id_2_idx = pd.NamedAgg('id_2_idx', agg_rand)
        )
        node2_nb = sample_df.loc[sample_df['id_1_idx'].isin(pos_node_df.id_2_idx.unique()), 'id_2_idx'].unique().tolist()
        neg_df = sample_df[
            (~sample_df.id_1_idx.isin(
                pos_node_df['id_1_idx'].tolist() + pos_node_df['id_2_idx'].tolist() + node2_nb
            )) & (~sample_df.id_2_idx.isin(
                pos_node_df['id_1_idx'].tolist() + pos_node_df['id_2_idx'].tolist() + node2_nb
            ))
        ] 
        rp = neg_df['id_1_idx'].nunique() < sample_node
        pos_node_df['neg'] = np.random.choice(neg_df['id_1_idx'].unique().tolist(), sample_node, replace=rp).tolist()
        return torch.LongTensor(pos_node_df.values)

    def _random_edge_sample(self):
        data_list = []
        for _ in tqdm(range(self.random_graph_count), total=self.random_graph_count):
            edges_smaple = edges.sample(frac=self.random_sample_ratio, replace=False).reset_index(drop=True)
            data = Data(
                x=self.node_features, 
                edge_index=torch.LongTensor(edges_smaple[['id_1_idx', 'id_2_idx']].values.T),
                y=self.node_label,
                mask=self.mask,
                x_info=self.node_info,
                sim=self.similar_sample(edges_smaple)
            )
            data_list.append(data)
        return data_list

    def _random_node_sample(self):
        data_list = []
        # random node sample 
        half1 = self.random_graph_count //  2
        for _ in tqdm(range(half1), total=half1):
            sample_node = np.random.choice(self.node_info, size=int(len(self.node_info) * self.random_sample_ratio), replace=False)
            nb1 = edges.loc[edges.id_1.isin(sample_node), 'id_1'].unique().tolist()
            tt_node = list(set(sample_node) | set(nb1))
            edges_smaple = edges[edges.id_1.isin(tt_node)].reset_index(drop=True)
            data = Data(
                x=self.node_features, 
                edge_index=torch.LongTensor(edges_smaple[['id_1_idx', 'id_2_idx']].values.T),
                y=self.node_label,
                mask=self.mask,
                x_info=self.node_info,
                sim=self.similar_sample(edges_smaple)
            )
            data_list.append(data)
            
        half2 = self.random_graph_count - half1
        for _ in tqdm(range(half2), total=half2):
            sample_node = np.random.choice(self.node_info, size=int(len(self.node_info) * self.random_sample_ratio), replace=False)
            nb1 = edges.loc[edges.id_2.isin(sample_node), 'id_2'].unique().tolist()
            tt_node = list(set(sample_node) | set(nb1))
            edges_smaple = edges[edges.id_2.isin(tt_node)].reset_index(drop=True)
            data = Data(
                x=self.node_features, 
                edge_index=torch.LongTensor(edges_smaple[['id_1_idx', 'id_2_idx']].values.T),
                y=self.node_label,
                mask=self.mask,
                x_info=self.node_info,
                sim=self.similar_sample(edges_smaple)
            )
            data_list.append(data)
        return data_list

    def process(self):
        data_list = []
        if self.idx == 2:
            edges_smaple = edges.reset_index(drop=True).copy(deep=True)
            data = Data(
                x=self.node_features, 
                edge_index=torch.LongTensor(edges_smaple[['id_1_idx', 'id_2_idx']].values.T),
                y=self.node_label,
                mask=self.mask,
                x_info=self.node_info,
                sim=self.similar_sample(edges_smaple, sample_node=500)
            )
            data_list.append(data)
            data, slices = self.collate(data_list)
            torch.save((data, slices), self.processed_paths[self.idx])  
            return None
        
        if self.sample_method == 'randomEdge':
            data_list = self._random_edge_sample()
        else:
            data_list = self._random_node_sample()

        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[self.idx])   
        return None


In [5]:
# dataloader
tr_dataset = graphData('./data/', label_df, data_type='train', random_sample_ratio=0.3, random_graph_count=256)
val_dataset = graphData('./data/', label_df, data_type='val', random_sample_ratio=0.3, random_graph_count=64)
te_dataset = graphData('./data/', label_df, data_type='test', random_sample_ratio=0.3, random_graph_count=64)


# Model 

In [6]:
class GAT(torch.nn.Module):
    def __init__(self, num_emb, emb_dim=128, heads=8, hidden_c=16, dropout=0.6):
        super().__init__()
        self.node_emb = nn.Embedding(num_embeddings=num_emb, embedding_dim=emb_dim)
        self.conv1 = GATConv(emb_dim, hidden_c, heads, dropout=dropout)
        self.conv4 = GATConv(hidden_c * heads, 2, heads=1,
                             concat=False, dropout=dropout)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        # [b, n, emb_dim]
        x = self.node_emb(x)
        x = x.squeeze(1) 
        
        x = F.dropout(x, p=0.2, training=self.training)
        x = F.relu(self.conv1(x, edge_index))

        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv4(x, edge_index)
        return x
    
    def similary(self, sim):
        id1, id2, neg  = sim[:, 0], sim[:, 1], sim[:, 2]
        id1 = self.node_emb(id1)
        id2 = self.node_emb(id2)
        neg = self.node_emb(neg)
        sc = torch.mul(id1, id2).sum(dim=1)
        sc = torch.clamp(sc, min=-10, max=10)
        sc = F.logsigmoid(sc)
        
        neg_sc = torch.mul(id1, neg).sum(dim=1)
        neg_sc = torch.clamp(neg_sc, min=-10, max=10)
        neg_sc = F.logsigmoid(-1 * neg_sc)
        loss = -(torch.sum(sc) + torch.sum(neg_sc))
        return loss


class GATResidule(torch.nn.Module):
    def __init__(self, num_emb, emb_dim=128, heads=8, hidden_c=16, dropout=0.6):
        super().__init__()
        self.node_emb = nn.Embedding(num_embeddings=num_emb, embedding_dim=emb_dim)
        self.conv1 = GATConv(emb_dim, hidden_c, heads, dropout=dropout)
        self.conv4 = GATConv(hidden_c * heads, 2, heads=1, concat=False, dropout=dropout)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        # [b, n, emb_dim]
        x = self.node_emb(x)
        x = x_org = x.squeeze(1) 
        
        x = F.dropout(x, p=0.2, training=self.training)
        x = x1 = F.relu(self.conv1(x, edge_index))
        
        # final layer add residule
        x = x_org + x1
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv4(x, edge_index)
        return x
    
    def similary(self, sim):
        id1, id2, neg  = sim[:, 0], sim[:, 1], sim[:, 2]
        id1 = self.node_emb(id1)
        id2 = self.node_emb(id2)
        neg = self.node_emb(neg)
        sc = torch.mul(id1, id2).sum(dim=1)
        sc = torch.clamp(sc, min=-10, max=10)
        sc = F.logsigmoid(sc)
        
        neg_sc = torch.mul(id1, neg).sum(dim=1)
        neg_sc = torch.clamp(neg_sc, min=-10, max=10)
        neg_sc = F.logsigmoid(-1 * neg_sc)
        loss = -(torch.sum(sc) + torch.sum(neg_sc))
        return loss


class GATAGNN(torch.nn.Module):
    def __init__(self, num_emb, emb_dim=128, heads=8, hidden_c=16, dropout=0.6):
        super().__init__()
        self.node_emb = nn.Embedding(num_embeddings=num_emb, embedding_dim=emb_dim)
        self.conv1 = GATConv(emb_dim, hidden_c, heads, dropout=dropout)
        self.conv2 = GATConv(hidden_c * heads, hidden_c, heads, dropout=dropout)
        self.a_conv = AGNNConv()
        self.conv4 = GATConv(hidden_c * heads, 2, heads=1,
                             concat=False, dropout=dropout)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        # [b, n, emb_dim]
        x = self.node_emb(x)
        x = x.squeeze(1) 
        
        x = F.dropout(x, p=0.2, training=self.training)
        x = F.relu(self.conv1(x, edge_index))
        
        x = F.dropout(x, p=0.2, training=self.training)
        x = F.relu(self.conv2(x, edge_index))
        
        x = self.a_conv(x, edge_index)
        
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv4(x, edge_index)
        return x
    

class gcn(nn.Module):
    def __init__(self, num_emb, emb_dim=128, hidden_c=16, drop_out=0.5):
        super().__init__()
        self.drop_out = drop_out
        self.node_emb = nn.Embedding(num_embeddings=num_emb, embedding_dim=emb_dim)
        self.conv1 = GCNConv(emb_dim, hidden_c, cached=False)
        self.conv3 = GCNConv(hidden_c, 2, cached=False)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        # [b, n, emb_dim]
        x = self.node_emb(x)
        x = x.squeeze(1) 
        x = F.dropout(x, p=self.drop_out, training=self.training)
        x = self.conv1(x, edge_index, None).relu()
        x = F.dropout(x, p=self.drop_out, training=self.training)
        x = self.conv3(x, edge_index, None)
        return x
    
    def similary(self, sim):
        id1, id2, neg  = sim[:, 0], sim[:, 1], sim[:, 2]
        id1 = self.node_emb(id1)
        id2 = self.node_emb(id2)
        neg = self.node_emb(neg)
        sc = torch.mul(id1, id2).sum(dim=1)
        sc = torch.clamp(sc, min=-10, max=10)
        sc = F.logsigmoid(sc)
        
        neg_sc = torch.mul(id1, neg).sum(dim=1)
        neg_sc = torch.clamp(neg_sc, min=-10, max=10)
        neg_sc = F.logsigmoid(-1 * neg_sc)
        loss = -(torch.sum(sc) + torch.sum(neg_sc))
        return loss


# Triaining 

In [11]:

def trainer(train_loader, valid_loader, model, config, similary_resample=False):
    device = config.device
    model.to('cpu')
    model.to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=config.lr) 
    save_path = config.save_path

    if not os.path.isdir('./models'):
        os.mkdir('./models')

    wandb.login()
    cfg_dict = config.__dict__
    wandb.init(
        project="XUNFEI-social-GCN-Strong-0929",
        config=cfg_dict
    )

    n_epochs, best_loss, step, early_stop_count = config.n_epochs, math.inf, 0, 0
    ep_val_loss_collect = []
    for epoch in range(n_epochs):
        gc.collect()
        torch.cuda.empty_cache()
        model.train()
        loss_record = []
        train_pbar = tqdm(train_loader, position=0, leave=True)
        train_pbar.set_description(f'Epoch [{epoch+1:02d}/{n_epochs:02d} ]')
        ep_right_count = 0
        tt_count = 0
        t_cnt = 0
        tp_cnt = 0
        for data in train_pbar:
            optimizer.zero_grad() 
            data = data.to(device)
            label = data.y.to(device)
            pred = model(data)
            mask = data.mask.to(device)
            loss = loss_fn(pred[mask], label[mask].long())
            sim_data = data.sim.to(device)
            if similary_resample and hasattr(train_loader, "similar_sample"):
                sim_data = train_loader.similar_sample(edges).to(device)

            sim_loss = model.similary(sim_data) / 100.0 / config.batch_size
            # loss_tt = 0.65 * loss + 0.35 * (sim_loss / 20.0) # 0.820
            loss_tt = (1 - config.similar_weight) * loss +  config.similar_weight * (sim_loss / 20.0) # 0.820
            tt_count += label[mask].size(0)
            ep_right_count += torch.sum(pred[mask].argmax(dim=1) == label[mask]).cpu().detach().item()
            loss_tt.backward()
            optimizer.step()
            
            # # similary
            # optimizer.zero_grad() 
            # sim_loss = model.similary(data.sim.to(device))
            # sim_loss.backward()
            # optimizer.step()

            step += 1
            sim_l_ = sim_loss.detach().item()
            l_ = loss.detach().item()
            loss_record.append(l_)
            t_cnt += torch.sum(1.0 * (pred[mask].argmax(dim=1) == 1)).cpu().detach().item()
            tp_cnt += torch.sum((pred[mask].argmax(dim=1) == 1) & (label[mask] == 1)).cpu().detach().item()
            p_ = tp_cnt / (t_cnt + 1e-5) * 100
            train_pbar.set_postfix({
                'loss': f'{l_:.5f}', 
                'acc': '{:.5f}'.format(ep_right_count/tt_count), 
                'percision': '{:.2f}%'.format(p_),
                'simLoss': '{:.5f}'.format(sim_l_)
            })
        
        mean_train_loss = sum(loss_record)/len(loss_record)
        mean_train_acc = ep_right_count/tt_count * 100
        tr_percision = tp_cnt / (t_cnt + 1e-5) * 100
        model.eval()
        loss_record = []
        ep_right_count = 0
        tt_count = 0
        t_cnt = 0
        tp_cnt = 0
        for data in valid_loader:
            data = data.to(device)
            label = data.y.to(device)
            with torch.no_grad():
                pred = model(data)
                mask = data.mask.to(device)
                loss = loss_fn(pred[mask], label[mask].long())
                tt_count += label[mask].size(0)
                ep_right_count += torch.sum(pred[mask].argmax(dim=1) == label[mask]).cpu().detach().item()
                
                t_cnt += torch.sum((1* (pred[mask].argmax(dim=1) == 1)).float()).cpu().detach().item()
                tp_cnt += torch.sum((pred[mask].argmax(dim=1) == 1) & (label[mask] == 1)).cpu().detach().item()

            loss_record.append(loss.item())
        
        val_percision = tp_cnt / (t_cnt + 1e-5) * 100
        mean_valid_loss = sum(loss_record)/len(loss_record)
        mean_valid_acc = ep_right_count/tt_count * 100
        ep_val_loss_collect.append(mean_valid_loss)
        print(f"""Epoch [{epoch+1:02d}/{n_epochs:02d}]: 
                    Train loss: {mean_train_loss:.4f} Acc: {mean_train_acc:.2f}% Percision: {tr_percision:.2f}%
                    Valid loss: {mean_valid_loss:.4f} Acc: {mean_valid_acc:.2f}% Percision: {val_percision:.2f}%
        """)
        log_dict = {
            "TrainLoss": mean_train_loss,
            'TrainAcc': mean_train_acc,
            "TrainPercision": tr_percision,
            'ValLoss': mean_valid_loss,
            "ValAcc": mean_valid_acc,
            "ValPercision": val_percision,
            "bestLoss": best_loss
        }
        wandb.log(log_dict)
        if (epoch+1) % 10 == 0:
            dt_hhmm = datetime.now().strftime('%Y%m%d_%H%M')
            save_path_f = save_path.replace('.ckpt', f'_10ep_{dt_hhmm}.ckpt')
            torch.save(model.state_dict(), save_path_f) 
        
        if mean_valid_loss < best_loss:
            best_loss = mean_valid_loss
            torch.save(model.state_dict(), save_path) 
            print('Saving model with loss {:.3f}...'.format(best_loss))
            early_stop_count = 0
        else: 
            early_stop_count += 1


        if early_stop_count >= config.early_stop:
            print('\nModel is not improving, so we halt the training session.')
            break
    
    wandb.finish()
    return ep_val_loss_collect

In [12]:
gc.collect()
torch.cuda.empty_cache()

In [13]:
config = Namespace(
    device='cuda',
    n_epochs=30,
    save_path='./models/gcn-strong.ckpt',
    lr=5e-4,
    batch_size=6,
    early_stop=100,
    similar_weight=0.3,
    
    emb_dim=256,
    hidden_c=256,
    drop_out=0.5
)
train_loader = DataLoader(tr_dataset, batch_size=config.batch_size, shuffle=True)
valid_loader = DataLoader(val_dataset, batch_size=config.batch_size, shuffle=False)

In [14]:
# model
gcn_model = gcn(num_emb=len(total_node), emb_dim=config.emb_dim, hidden_c=config.hidden_c)
gcn_loss_record = trainer(train_loader, valid_loader, gcn_model, config)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: 296294812. Use `wandb login --relogin` to force relogin


Epoch [01/30 ]: 100%|██████████| 43/43 [00:02<00:00, 15.25it/s, loss=0.76084, acc=0.63304, percision=28.11%, simLoss=15.07732]


Epoch [01/30]: 
                    Train loss: 0.8694 Acc: 63.30% Percision: 28.11%
                    Valid loss: 0.5741 Acc: 73.78% Percision: 48.70%
        
Saving model with loss 0.574...


Epoch [02/30 ]: 100%|██████████| 43/43 [00:02<00:00, 19.43it/s, loss=0.70171, acc=0.67759, percision=32.45%, simLoss=13.72178]


Epoch [02/30]: 
                    Train loss: 0.7306 Acc: 67.76% Percision: 32.45%
                    Valid loss: 0.5388 Acc: 75.10% Percision: 69.86%
        
Saving model with loss 0.539...


...

Epoch [29/30]: 
                    Train loss: 0.2064 Acc: 92.19% Percision: 87.87%
                    Valid loss: 0.1440 Acc: 94.86% Percision: 92.20%
        
Saving model with loss 0.144...


Epoch [30/30 ]: 100%|██████████| 43/43 [00:02<00:00, 19.36it/s, loss=0.19879, acc=0.92411, percision=88.22%, simLoss=9.81712] 


Epoch [30/30]: 
                    Train loss: 0.2012 Acc: 92.41% Percision: 88.22%
                    Valid loss: 0.1412 Acc: 94.95% Percision: 92.26%
        
Saving model with loss 0.141...


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


TrainAcc,▁▂▂▃▃▃▄▄▄▅▅▅▆▆▆▆▇▇▇▇▇▇████████
TrainLoss,█▇▆▆▅▅▅▄▄▄▄▃▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁
TrainPercision,▁▂▂▂▃▃▄▅▅▅▆▆▆▆▇▇▇▇▇▇▇█████████
ValAcc,▁▁▂▂▃▃▄▄▅▅▆▆▆▇▇▇▇▇▇███████████
ValLoss,█▇▇▇▆▆▆▅▅▅▄▄▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁
ValPercision,▁▄▆▇▇▇██▇█████████████████████
bestLoss,█▇▇▇▆▆▆▅▅▄▄▄▃▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁
TrainAcc,92.41093
TrainLoss,0.20123
TrainPercision,88.21668
ValAcc,94.9498


In [15]:
config_gcn2 = Namespace(
    device='cuda',
    n_epochs=600,
    save_path='./models/gcn2-strong.ckpt',
    lr=5e-4,
    batch_size=6,
    early_stop=100,
    similar_weight=0.3,
    
    emb_dim=256,
    hidden_c=256,
    drop_out=0.5
)

gcn_model2 = gcn(num_emb=len(total_node), emb_dim=config_gcn2.emb_dim, hidden_c=config_gcn2.hidden_c)
gcn_loss_record2 = trainer(te_dataset, valid_loader, gcn_model2, config_gcn2, similary_resample=True)

Epoch [01/600 ]: 100%|██████████| 1/1 [00:00<00:00, 17.34it/s, loss=0.76804, acc=0.63090, percision=28.76%, simLoss=3.38438]


Epoch [01/600]: 
                    Train loss: 0.7680 Acc: 63.09% Percision: 28.76%
                    Valid loss: 0.6628 Acc: 67.02% Percision: 29.65%
        
Saving model with loss 0.663...


Epoch [02/600 ]: 100%|██████████| 1/1 [00:00<00:00, 22.10it/s, loss=0.76349, acc=0.64101, percision=27.05%, simLoss=3.19517]


Epoch [02/600]: 
                    Train loss: 0.7635 Acc: 64.10% Percision: 27.05%
                    Valid loss: 0.6515 Acc: 68.74% Percision: 30.52%
        
Saving model with loss 0.651...


Epoch [03/600 ]: 100%|██████████| 1/1 [00:00<00:00, 21.29it/s, loss=0.74292, acc=0.66220, percision=29.95%, simLoss=3.02798]


...

Epoch [599/600]: 
                    Train loss: 0.3022 Acc: 87.72% Percision: 81.43%
                    Valid loss: 0.2888 Acc: 88.43% Percision: 78.40%
        


Epoch [600/600 ]: 100%|██████████| 1/1 [00:00<00:00, 20.15it/s, loss=0.30211, acc=0.87664, percision=81.35%, simLoss=2.88357]


Epoch [600/600]: 
                    Train loss: 0.3021 Acc: 87.66% Percision: 81.35%
                    Valid loss: 0.2887 Acc: 88.46% Percision: 78.37%
        


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


TrainAcc,▁▁▁▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇███████
TrainLoss,█▇▇▇▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁
TrainPercision,▁▁▂▂▂▃▄▄▄▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇███████████
ValAcc,▁▁▁▂▂▂▂▃▃▃▃▃▄▄▄▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇███████
ValLoss,██▇▇▇▆▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁
ValPercision,▁▁▂▃▄▄▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇████████████████
bestLoss,██▇▇▇▆▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁
TrainAcc,87.66426
TrainLoss,0.30211
TrainPercision,81.34647
ValAcc,88.45961


In [16]:
config_gcn3 = Namespace(
    device='cuda',
    n_epochs=600,
    save_path='./models/gcn3-strong.ckpt',
    lr=5e-4,
    batch_size=6,
    early_stop=100,
    similar_weight=0.3,
    
    emb_dim=512,
    hidden_c=256,
    drop_out=0.5
)

gcn_model3 = gcn(num_emb=len(total_node), emb_dim=config_gcn3.emb_dim, hidden_c=config_gcn3.hidden_c)
gcn_loss_record2 = trainer(te_dataset, valid_loader, gcn_model3, config_gcn3, similary_resample=True)

Epoch [01/600 ]: 100%|██████████| 1/1 [00:00<00:00, 18.12it/s, loss=0.89822, acc=0.54563, percision=27.11%, simLoss=4.12385]


Epoch [01/600]: 
                    Train loss: 0.8982 Acc: 54.56% Percision: 27.11%
                    Valid loss: 0.7441 Acc: 57.74% Percision: 27.51%
        
Saving model with loss 0.744...


Epoch [02/600 ]: 100%|██████████| 1/1 [00:00<00:00, 18.43it/s, loss=0.85952, acc=0.56404, percision=27.19%, simLoss=3.89636]


Epoch [02/600]: 
                    Train loss: 0.8595 Acc: 56.40% Percision: 27.19%
                    Valid loss: 0.7121 Acc: 61.43% Percision: 28.62%
        
Saving model with loss 0.712...


Epoch [03/600 ]: 100%|██████████| 1/1 [00:00<00:00, 19.93it/s, loss=0.82376, acc=0.60451, percision=28.20%, simLoss=3.83601]


...

Epoch [599/600]: 
                    Train loss: 0.2553 Acc: 89.88% Percision: 84.68%
                    Valid loss: 0.2540 Acc: 90.02% Percision: 80.20%
        
Saving model with loss 0.254...


Epoch [600/600 ]: 100%|██████████| 1/1 [00:00<00:00, 19.09it/s, loss=0.24745, acc=0.90235, percision=85.42%, simLoss=3.62012]


Epoch [600/600]: 
                    Train loss: 0.2474 Acc: 90.23% Percision: 85.42%
                    Valid loss: 0.2540 Acc: 90.02% Percision: 80.18%
        


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


TrainAcc,▁▂▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇███████
TrainLoss,██▇▆▆▆▅▅▅▄▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁
TrainPercision,▁▂▂▃▃▄▄▄▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇███████████
ValAcc,▁▂▂▂▃▃▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇████████████
ValLoss,█▇▇▆▆▆▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁
ValPercision,▁▂▃▄▅▅▆▆▆▇▇▇▇▇▇▇▇▇██████████████████████
bestLoss,█▇▇▆▆▆▅▅▅▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁
TrainAcc,90.23466
TrainLoss,0.24745
TrainPercision,85.42383
ValAcc,90.02144


In [17]:
config_gcn4 = Namespace(
    device='cuda',
    n_epochs=600,
    save_path='./models/gcn2-strong.ckpt',
    lr=5e-4,
    batch_size=6,
    early_stop=100,
    similar_weight=0.15,
    
    emb_dim=256,
    hidden_c=256,
    drop_out=0.5
)

gcn_model4 = gcn(num_emb=len(total_node), emb_dim=config_gcn4.emb_dim, hidden_c=config_gcn4.hidden_c)
gcn_loss_record2 = trainer(te_dataset, valid_loader, gcn_model4, config_gcn4, similary_resample=True)

Epoch [01/600 ]: 100%|██████████| 1/1 [00:00<00:00, 20.31it/s, loss=1.02590, acc=0.41711, percision=25.47%, simLoss=3.46754]


Epoch [01/600]: 
                    Train loss: 1.0259 Acc: 41.71% Percision: 25.47%
                    Valid loss: 0.8373 Acc: 44.94% Percision: 26.15%
        
Saving model with loss 0.837...


Epoch [02/600 ]: 100%|██████████| 1/1 [00:00<00:00, 21.06it/s, loss=0.96395, acc=0.44513, percision=25.42%, simLoss=3.23745]


Epoch [02/600]: 
                    Train loss: 0.9639 Acc: 44.51% Percision: 25.42%
                    Valid loss: 0.7918 Acc: 49.39% Percision: 26.82%
        
Saving model with loss 0.792...


Epoch [03/600 ]: 100%|██████████| 1/1 [00:00<00:00, 21.12it/s, loss=0.90207, acc=0.49671, percision=26.16%, simLoss=3.43309]


...

Epoch [599/600]: 
                    Train loss: 0.2898 Acc: 88.19% Percision: 82.24%
                    Valid loss: 0.2654 Acc: 89.66% Percision: 80.08%
        
Saving model with loss 0.265...


Epoch [600/600 ]: 100%|██████████| 1/1 [00:00<00:00, 20.96it/s, loss=0.28686, acc=0.88419, percision=82.51%, simLoss=3.22938]


Epoch [600/600]: 
                    Train loss: 0.2869 Acc: 88.42% Percision: 82.51%
                    Valid loss: 0.2652 Acc: 89.67% Percision: 80.08%
        
Saving model with loss 0.265...


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


TrainAcc,▁▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇███████
TrainLoss,█▇▇▆▆▆▆▆▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁
TrainPercision,▁▁▂▂▃▃▃▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇████████
ValAcc,▁▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇████████
ValLoss,██▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁
ValPercision,▁▁▂▃▄▄▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇███████████████
bestLoss,█▇▇▇▆▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁
TrainAcc,88.41877
TrainLoss,0.28686
TrainPercision,82.51251
ValAcc,89.67046


In [22]:
# config_gat = Namespace(
#     device='cuda',
#     n_epochs=512,
#     save_path='./models/gat-strong.ckpt',
#     lr=5e-4,
#     batch_size=6,
#     early_stop=100,
#     similar_weight=0.3,

#     emb_dim=256,
#     heads=8,
#     hidden_c=32,
#     dropout=0.5 # 0.6
# )

# gat_model = GAT(num_emb=len(total_node), emb_dim=config_gat.emb_dim, heads=config_gat.heads, hidden_c=config_gat.hidden_c, dropout=config_gat.dropout)
# gat_loss_record = trainer(te_dataset, valid_loader, gat_model, config_gat, similary_resample=True)

# ensemble

In [19]:
pred_list = []
for model in [gcn_model, gcn_model2, gcn_model3, gcn_model4]:
    gc.collect()
    torch.cuda.empty_cache()
    model.eval()
    device = config.device
    loss_record = []
    test_accs = []
    ep_right_count = 0
    tt_count = 0
    data = te_dataset[0]
    data = data.to(device)
    label = data.y.to(device)
    with torch.no_grad():
        pred = model(data)
        pred_list.append(pred.cpu().detach().numpy())
        mask = data.mask
        tt_count += label[mask].size(0)
        ep_right_count += torch.sum(pred[mask].argmax(dim=1) == label[mask]).cpu().detach().item()

    t_cnt, tp_cnt = 0, 0
    t_cnt += torch.sum(1.0 * (pred[mask].argmax(dim=1) == 1)).cpu().detach().item()
    tp_cnt += torch.sum((pred[mask].argmax(dim=1) == 1) & (label[mask] == 1)).cpu().detach().item()
    acc_ = torch.sum(pred[mask].argmax(dim=1) == label[mask]).cpu().detach().item()
    acc_ /= torch.sum(mask).cpu().detach().item()
    p_ = tp_cnt / (t_cnt + 1e-5)

    print(t_cnt, tp_cnt, p_, acc_)

6389.0 5855 0.9164188434552843 0.9309025270758122
6359.0 5675 0.8924359161936847 0.9189891696750903
6790.0 6124 0.9019145789367974 0.9358483754512635
6571.0 5876 0.8942322311760277 0.9258483754512635


In [23]:
pred = pred_list[0] + pred_list[1] + pred_list[2] #+ pred_list[3]
pred

array([[  1.3151   ,  -2.870316 ],
       [  3.8984065,  -5.538137 ],
       [  3.8642468,  -2.8128047],
       ...,
       [  0.5328629,  -1.95279  ],
       [  8.029915 , -11.565819 ],
       [  4.240053 ,  -5.322579 ]], dtype=float32)

## submit

In [24]:
res = pd.DataFrame({
'id':  data.x_info,
'target': pred.argmax(axis=1)
})
submit_df = pd.read_csv('./data/sample_submit.csv')
sub = submit_df[['id']].merge(res, how='left', on='id')
sub['target'].isna().value_counts()
print(sub['target'].value_counts())
# 0.820
# sub.to_csv('./data/sub_strong_0923_02_org.csv', index=False)

sub.to_csv('./data/sub_strong_gcn_gcn2_gcn3_0929_01_org.csv', index=False)

0    8062
1    1938
Name: target, dtype: int64


# Supervised Learning 

In [20]:
## get features
model.eval()
data = te_dataset[0]
data = data.to(device)
x, edge_index, batch = data.x, data.edge_index, data.batch
# [b, n, emb_dim]
x = model.node_emb(x)
x_emb = x
x = x.squeeze(1) 
x = F.relu(model.conv1(x, edge_index))
x = F.relu(model.conv2(x, edge_index))
# x = model.a_conv(x, edge_index)

mask = data.mask.cpu().detach().numpy()
x_tt = x.cpu().detach().numpy()
# x_tt = np.c_[x_tt, x_emb.cpu().detach().numpy()]
x_tt += x_emb.cpu().detach().numpy()
# x_tt = x_emb.cpu().detach().numpy()
x_tr = x_tt[mask]
x_te = x_tt[~mask]
print(x_tr.shape, x_tt.shape, x_te.shape)
y_tt = data.y.cpu().detach().numpy()
y_tr = y_tt[mask]
y_te = y_tt[~mask]
print(y_tt.shape, y_tr.shape, y_te.shape)

(27700, 256) (37700, 256) (10000, 256)
(37700,) (27700,) (10000,)


In [73]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold

kf = KFold(5)
final_res = np.zeros((x_te.shape[0], 2))

features = None
for i, (train_index, test_index) in enumerate(kf.split(x_tr)):
    x_tr_i, y_tr_i = x_tr[train_index], y_tr[train_index]
    x_val_i, y_val_i = x_tr[test_index], y_tr[test_index]
    if features is not None:
        x_tr_i = x_tr_i[:, features]
        x_val_i = x_val_i[:, features]

    lr = RandomForestClassifier(n_estimators=120, max_depth=8)
    lr.fit(x_tr_i, y_tr_i)
    tr_pred_i = lr.predict(x_val_i)
    acc = accuracy_score(y_val_i, tr_pred_i)
    
    final_res += lr.predict_proba(x_te if features is None else x_te[:, features])
    print(f'[ fold {i+1} / 5 ] acc={acc*100:.2f}%')
    if i == 0:
        # features = np.argsort(lr.feature_importances_)[:100]
        continue


[ fold 1 / 5 ] acc=98.07%
[ fold 2 / 5 ] acc=97.94%
[ fold 3 / 5 ] acc=97.67%
[ fold 4 / 5 ] acc=97.92%
[ fold 5 / 5 ] acc=98.18%


In [77]:
res = pd.DataFrame({
    'id':  data.x_info[~mask],
    'target': final_res.argmax(axis=1) 
})
print(res['target'].value_counts())

0    7919
1    2081
Name: target, dtype: int64


## submit

In [78]:
submit_df = pd.read_csv('./data/sample_submit.csv')
sub = submit_df[['id']].merge(res, how='left', on='id')
sub['target'].value_counts()

0    7919
1    2081
Name: target, dtype: int64

In [79]:
sub.to_csv('./data/sub_strong_0923_01.csv', index=False)